In [24]:
import matplotlib

matplotlib.use('TkAgg')
import LDP.basicDP.OLHbasic as lhb
import LDP.basicFunction.basicfunc as bf
import LDP.basicDP.LDPKVbasic as lkvb
import LDP.basicDP.MLPKVbasic as mlkvp
import numpy as np
import random
import time


"""数据处理"""
# 关闭科学计数法显示
np.set_printoptions(suppress=True)

##### 读取文件
##### 构建kv对元组列表，每一行代表一个用户的kv对list

In [25]:
# k_path='../LDPdataset/Clothing/data/Clothing_k.txt'
# v_path='../LDPdataset/Clothing/data/Clothing_v.txt'
# lable_path='../LDPdataset/Clothing/data/Clothing_lable.txt'
data_name='Clothing'

k_path='../MLPKV/data/'+data_name+'/'+data_name+'_k.txt'
v_path='../MLPKV/data/'+data_name+'/'+data_name+'_v.txt'
true_path='../MLPKV/data/'+data_name+'/'+data_name+'_id_count_avg_norm.txt'


data_k = bf.readtxt(k_path)
data_v = bf.readtxt(v_path)
true_count= bf.readtxt(true_path)

n = len(data_k)  # 用户数量
d = len(true_count)  # key域的长度
epsilon = 5

# 构建kv对元组列表，每一行代表一个用户的kv对list
kv = []

for i in range(n):
    data_k[i]=list(map(int,data_k[i]))
    tmp = zip(data_k[i], data_v[i])
    kv.append(list(tmp))

# 构建label标签，我已经把标签规范化为[1,l]的数字了，也就是说不需要用之前的原始标签了,不用查字典了
label=[i+1 for i in range(d)]
kv

[[(213, 1.0), (92, 1.0)],
 [(4007, 1.0)],
 [(3029, 1.0), (2492, 1.0)],
 [(2125, 1.0)],
 [(20, 0.6), (48, 1.0), (1244, 1.0), (2696, 0.6), (58, 0.6)],
 [(1973, 1.0)],
 [(2261, 1.0)],
 [(3867, 1.0)],
 [(481, 1.0)],
 [(2084, 1.0)],
 [(1707, 0.6)],
 [(30, 1.0)],
 [(3601, 1.0)],
 [(57, 0.6), (11, 1.0)],
 [(2625, 0.6)],
 [(613, 1.0), (3181, 1.0)],
 [(2021, 1.0)],
 [(1475, 1.0), (1777, 0.6)],
 [(651, 1.0)],
 [(3815, 0.6)],
 [(98, 0.2), (12, 1.0), (690, 1.0)],
 [(70, 0.6), (3867, 0.6)],
 [(550, 1.0)],
 [(683, 1.0), (822, 1.0), (11, 1.0), (28, 1.0), (103, 1.0)],
 [(3, 1.0)],
 [(1178, 1.0)],
 [(892, 1.0), (2750, 1.0), (854, 1.0), (2309, 1.0)],
 [(1034, 1.0)],
 [(1037, 0.6)],
 [(99, 1.0)],
 [(1698, 1.0)],
 [(25, 1.0)],
 [(63, 1.0)],
 [(2521, 1.0)],
 [(1958, 1.0), (902, 1.0)],
 [(1475, 1.0), (1271, 1.0)],
 [(3297, 1.0), (3452, 0.6)],
 [(8, 0.6)],
 [(2163, 1.0)],
 [(1, 1.0)],
 [(3328, 1.0), (1826, 0.6)],
 [(1029, 0.6)],
 [(75, -0.2)],
 [(1538, 1.0)],
 [(16, 1.0)],
 [(1367, 1.0)],
 [(1607, 0.6)],
 [(

##### 分组，GroupA完成候选项集，GroupB完成填充项集，GroupC完成估计

In [26]:

GroupA,GroupB,GroupC=mlkvp.divide_list(kv,1,1,1)

n_A=len(GroupA) #A组用户数量
n_B=len(GroupB) #B组用户数量
n_C=len(GroupC) #C组用户数量


##### GroupA用户完成Step1和2
##### Step1：采样并扰动上传

In [27]:
GroupA

[[(213, 1.0), (92, 1.0)],
 [(4007, 1.0)],
 [(3029, 1.0), (2492, 1.0)],
 [(2125, 1.0)],
 [(20, 0.6), (48, 1.0), (1244, 1.0), (2696, 0.6), (58, 0.6)],
 [(1973, 1.0)],
 [(2261, 1.0)],
 [(3867, 1.0)],
 [(481, 1.0)],
 [(2084, 1.0)],
 [(1707, 0.6)],
 [(30, 1.0)],
 [(3601, 1.0)],
 [(57, 0.6), (11, 1.0)],
 [(2625, 0.6)],
 [(613, 1.0), (3181, 1.0)],
 [(2021, 1.0)],
 [(1475, 1.0), (1777, 0.6)],
 [(651, 1.0)],
 [(3815, 0.6)],
 [(98, 0.2), (12, 1.0), (690, 1.0)],
 [(70, 0.6), (3867, 0.6)],
 [(550, 1.0)],
 [(683, 1.0), (822, 1.0), (11, 1.0), (28, 1.0), (103, 1.0)],
 [(3, 1.0)],
 [(1178, 1.0)],
 [(892, 1.0), (2750, 1.0), (854, 1.0), (2309, 1.0)],
 [(1034, 1.0)],
 [(1037, 0.6)],
 [(99, 1.0)],
 [(1698, 1.0)],
 [(25, 1.0)],
 [(63, 1.0)],
 [(2521, 1.0)],
 [(1958, 1.0), (902, 1.0)],
 [(1475, 1.0), (1271, 1.0)],
 [(3297, 1.0), (3452, 0.6)],
 [(8, 0.6)],
 [(2163, 1.0)],
 [(1, 1.0)],
 [(3328, 1.0), (1826, 0.6)],
 [(1029, 0.6)],
 [(75, -0.2)],
 [(1538, 1.0)],
 [(16, 1.0)],
 [(1367, 1.0)],
 [(1607, 0.6)],
 [(

In [28]:
#从每个用户项集里随机采样一个key并扰动
sampling_perturbed_A=[] 
for i in range(n_A):
    random.seed(1)
    sampling_key=random.sample(GroupA[i],1)[0][0] #随机采样得到的项
    perturbed_key=lhb.OLH_perturbation(epsilon,sampling_key,i)
    sampling_perturbed_A.append((perturbed_key,i))
sampling_perturbed_A

[(135, 0),
 (75, 1),
 (66, 2),
 (30, 3),
 (12, 4),
 (46, 5),
 (64, 6),
 (135, 7),
 (11, 8),
 (104, 9),
 (94, 10),
 (83, 11),
 (74, 12),
 (81, 13),
 (99, 14),
 (120, 15),
 (42, 16),
 (131, 17),
 (54, 18),
 (6, 19),
 (35, 20),
 (75, 21),
 (88, 22),
 (125, 23),
 (13, 24),
 (61, 25),
 (5, 26),
 (62, 27),
 (146, 28),
 (0, 29),
 (117, 30),
 (41, 31),
 (78, 32),
 (144, 33),
 (6, 34),
 (41, 35),
 (59, 36),
 (71, 37),
 (148, 38),
 (99, 39),
 (29, 40),
 (36, 41),
 (105, 42),
 (90, 43),
 (12, 44),
 (102, 45),
 (31, 46),
 (141, 47),
 (25, 48),
 (18, 49),
 (113, 50),
 (122, 51),
 (5, 52),
 (138, 53),
 (12, 54),
 (35, 55),
 (71, 56),
 (52, 57),
 (80, 58),
 (119, 59),
 (132, 60),
 (127, 61),
 (6, 62),
 (7, 63),
 (117, 64),
 (142, 65),
 (68, 66),
 (130, 67),
 (114, 68),
 (62, 69),
 (52, 70),
 (139, 71),
 (141, 72),
 (25, 73),
 (96, 74),
 (138, 75),
 (40, 76),
 (134, 77),
 (79, 78),
 (101, 79),
 (93, 80),
 (143, 81),
 (92, 82),
 (11, 83),
 (79, 84),
 (119, 85),
 (89, 86),
 (40, 87),
 (50, 88),
 (50, 89

##### Step2：聚合得到候选集

In [29]:
#Step1的估计结果

starttime1 = time.clock()
print("开始执行OLH...")
# 注意这里的用户数目n要用组A的用户数目，而不是所有的
est_A=lhb.OLH_aggregation_mutilprocess(epsilon,sampling_perturbed_A,n_A,d,lhb.get_support,8)
# 程序运行结束时间
endtime1 = time.clock()
print('运行时间:', endtime1 - starttime1)
est_A

开始执行OLH...
ForkPoolWorker-17 启动
ForkPoolWorker-18 启动
ForkPoolWorker-19 启动
ForkPoolWorker-20 启动
ForkPoolWorker-21 启动
ForkPoolWorker-22 启动
ForkPoolWorker-23 启动
ForkPoolWorker-24 启动
[35, 49, 36, 36, 32, 54, 38, 61, 30, 37, 39, 38, 39, 27, 47, 31, 49, 53, 33, 39, 43, 23, 48, 44, 26, 33, 37, 61, 32, 48, 48, 41, 25, 38, 35, 42, 24, 44, 35, 30, 38, 41, 36, 28, 28, 38, 28, 62, 41, 34, 33, 27, 34, 36, 30, 48, 39, 27, 34, 41, 24, 32, 37, 38, 32, 38, 40, 29, 33, 34, 20, 29, 35, 30, 33, 30, 31, 34, 25, 46, 37, 28, 25, 36, 36, 39, 32, 35, 43, 18, 33, 35, 42, 29, 30, 25, 33, 39, 37, 38, 48, 31, 30, 37, 27, 24, 35, 26, 31, 20, 25, 19, 28, 34, 26, 33, 31, 32, 28, 28, 29, 26, 27, 41, 30, 39, 39, 33, 20, 30, 27, 26, 38, 37, 26, 37, 36, 33, 29, 29, 28, 37, 30, 45, 34, 28, 30, 35, 25, 31, 40, 34, 36, 29, 34, 24, 34, 31, 27, 28, 25, 21, 22, 24, 34, 27, 28, 29, 26, 31, 36, 36, 29, 32, 32, 27, 31, 23, 33, 21, 27, 26, 24, 33, 33, 19, 14, 34, 39, 34, 29, 29, 29, 34, 25, 37, 29, 29, 23, 29, 35, 32, 37, 28, 41, 

[123.13215809432066,
 374.15166079217653,
 198.03313873803572,
 121.10780726611213,
 70.49903656089926,
 285.08022435100185,
 183.86268294057612,
 511.8075171103556,
 64.4259840762737,
 117.05910560969511,
 357.95685416650844,
 70.49903656089926,
 171.71657797132502,
 23.938967512103407,
 274.9584702099593,
 123.13215809432066,
 252.69061109966563,
 260.7880144124997,
 84.66949235835887,
 159.57047300207392,
 414.6386773563469,
 50.255528278814104,
 36.0850724813545,
 119.08345643790362,
 44.18247579418856,
 5.719810058226772,
 78.59643987373332,
 297.22632932025294,
 113.01040395327807,
 303.2993818048785,
 230.42275198937196,
 151.47306968923988,
 40.13377413777153,
 133.25391223536323,
 44.18247579418856,
 108.96170229686105,
 -2.3775932546072895,
 151.47306968923988,
 36.0850724813545,
 131.2295614071547,
 48.23117745060559,
 113.01040395327807,
 117.05910560969511,
 70.49903656089926,
 25.963318340311922,
 13.817213371060832,
 -4.401944082815804,
 471.32050054618526,
 131.22956140

In [30]:
#用做对比时间，这是没有用多线程的，上面是用了多进程的EC-OLH

# starttime2 = time.clock()
# print("开始执行OLH...")
# support=[]
# g = int(np.exp(epsilon)) + 1  # 参数g
# for i in range(d):
#     c=0
#     for j in range(len(sampling_perturbed_A)):
#         if(lhb.olh_hash(i+1,g,sampling_perturbed_A[j][1])==sampling_perturbed_A[j][0]):
#             c+=1
#     support.append(c)
# 
# #support
# 
# e=[]
# p = lhb.p_values(epsilon, n=g)  # 参数p
# 
# for i in range(len(support)):
#     es=lhb.aggregation(support[i],len(GroupA),g,p)
#     e.append(es)
# endtime2 = time.clock()
# print('运行时间:', endtime2 - starttime2)
# e

In [31]:
k=20
est_A_temp=list(zip(label,est_A))
est_A_temp.sort(key=lambda x:x[1],reverse=True)

 #候选集
candi_dict=dict(est_A_temp[:2*k])
Candidate=list(candi_dict.keys())

Candidate

[8,
 48,
 21,
 2,
 11,
 101,
 30,
 98,
 28,
 6,
 15,
 18,
 17,
 31,
 89,
 3,
 7,
 13,
 67,
 20,
 32,
 38,
 64,
 142,
 93,
 1314,
 34,
 57,
 40,
 49,
 1731,
 1973,
 1,
 16,
 3236,
 3867,
 4410,
 4,
 80,
 24]

##### GroupB用户完成Step3和4
##### Step3：取交集并扰动上传

In [32]:
GroupB

[[(966, 0.6)],
 [(1033, 1.0), (2955, 1.0)],
 [(48, 0.6)],
 [(3253, 1.0), (689, 1.0)],
 [(31, 1.0), (2006, 1.0)],
 [(2753, 1.0)],
 [(89, 0.6)],
 [(481, 1.0)],
 [(1918, 0.6)],
 [(43, 0.6), (5628, 0.6), (1682, 1.0)],
 [(8, 1.0)],
 [(2309, 1.0)],
 [(3759, 1.0)],
 [(1087, 1.0)],
 [(718, 0.6)],
 [(3783, 1.0)],
 [(5, 0.6)],
 [(1721, 1.0)],
 [(1078, 1.0),
  (3232, 0.6),
  (4355, 1.0),
  (2626, 1.0),
  (1292, 1.0),
  (1127, 0.2),
  (1153, 0.6),
  (512, 0.6),
  (3719, 1.0),
  (5095, 0.2),
  (3247, 1.0),
  (1497, 1.0),
  (3303, 1.0),
  (3004, 1.0),
  (220, 1.0),
  (1355, 1.0),
  (3367, 1.0),
  (1165, 1.0),
  (3065, 1.0),
  (2205, 1.0),
  (5153, 0.6),
  (1336, 1.0),
  (701, 0.6),
  (390, 1.0),
  (5486, 1.0),
  (3666, 0.6),
  (768, 0.6),
  (4641, 1.0),
  (4003, 0.6),
  (702, 0.6),
  (1992, 1.0),
  (3472, 0.6),
  (504, 0.6),
  (5118, 1.0),
  (237, 1.0)],
 [(58, 1.0), (3822, 1.0), (15, 1.0), (69, 1.0)],
 [(187, 1.0)],
 [(1973, 1.0)],
 [(31, 1.0)],
 [(66, 0.6), (93, 1.0), (2521, -0.2), (101, 0.6)],
 [

In [33]:
GroupB_k=[] #GroupB组用户拥有的k值
GroupB_num=[] #交集的个数
num_perturbed_B=[] #交集个数的扰动
# 获取GroupB用户拥有的k值
for i in range(n_B):
    each_k=list(zip(*GroupB[i]))[0]
    GroupB_k.append(each_k)
    num=len(set(Candidate).intersection(set(each_k)))
    GroupB_num.append(num)
    perturbed_num=lhb.OLH_perturbation(epsilon,num,i)
    num_perturbed_B.append((perturbed_num,i))
GroupB_k

[(966,),
 (1033, 2955),
 (48,),
 (3253, 689),
 (31, 2006),
 (2753,),
 (89,),
 (481,),
 (1918,),
 (43, 5628, 1682),
 (8,),
 (2309,),
 (3759,),
 (1087,),
 (718,),
 (3783,),
 (5,),
 (1721,),
 (1078,
  3232,
  4355,
  2626,
  1292,
  1127,
  1153,
  512,
  3719,
  5095,
  3247,
  1497,
  3303,
  3004,
  220,
  1355,
  3367,
  1165,
  3065,
  2205,
  5153,
  1336,
  701,
  390,
  5486,
  3666,
  768,
  4641,
  4003,
  702,
  1992,
  3472,
  504,
  5118,
  237),
 (58, 3822, 15, 69),
 (187,),
 (1973,),
 (31,),
 (66, 93, 2521, 101),
 (2786,),
 (5,),
 (2565,),
 (13,),
 (4289,
  2775,
  93,
  1813,
  3371,
  3769,
  1974,
  3642,
  5680,
  912,
  2692,
  4425,
  2220,
  307),
 (2718,),
 (18, 2460, 746),
 (21,),
 (2054, 2476),
 (3348, 1333, 300),
 (1702, 835),
 (5088,),
 (2525, 6, 2958),
 (64,),
 (101,),
 (1870,),
 (3094, 31, 3211, 1252),
 (2595,),
 (2,),
 (2538, 3234, 2176),
 (1857,),
 (2,),
 (2984, 2428, 2030, 3744, 261, 3914, 3403, 3739),
 (2847,),
 (916,),
 (8,),
 (61,),
 (30,),
 (1660,),
 (2

In [34]:
num_perturbed_B

[(98, 0),
 (56, 1),
 (133, 2),
 (111, 3),
 (97, 4),
 (116, 5),
 (35, 6),
 (96, 7),
 (53, 8),
 (99, 9),
 (109, 10),
 (127, 11),
 (137, 12),
 (75, 13),
 (138, 14),
 (3, 15),
 (99, 16),
 (44, 17),
 (53, 18),
 (145, 19),
 (71, 20),
 (34, 21),
 (138, 22),
 (51, 23),
 (50, 24),
 (130, 25),
 (48, 26),
 (66, 27),
 (25, 28),
 (94, 29),
 (45, 30),
 (18, 31),
 (129, 32),
 (126, 33),
 (143, 34),
 (64, 35),
 (105, 36),
 (50, 37),
 (94, 38),
 (40, 39),
 (130, 40),
 (0, 41),
 (146, 42),
 (6, 43),
 (121, 44),
 (55, 45),
 (81, 46),
 (131, 47),
 (0, 48),
 (51, 49),
 (123, 50),
 (141, 51),
 (66, 52),
 (13, 53),
 (47, 54),
 (78, 55),
 (50, 56),
 (93, 57),
 (38, 58),
 (32, 59),
 (110, 60),
 (100, 61),
 (123, 62),
 (58, 63),
 (62, 64),
 (140, 65),
 (28, 66),
 (67, 67),
 (30, 68),
 (144, 69),
 (10, 70),
 (69, 71),
 (30, 72),
 (50, 73),
 (60, 74),
 (101, 75),
 (137, 76),
 (18, 77),
 (29, 78),
 (41, 79),
 (144, 80),
 (12, 81),
 (132, 82),
 (93, 83),
 (63, 84),
 (28, 85),
 (51, 86),
 (97, 87),
 (83, 88),
 (102,

In [35]:
starttime3 = time.clock()
print("开始执行OLH...")
# 注意这里的用户数目n要用组B的用户数目，而不是所有的,并且数据维度也不是d了，而是[0,2k]，因为交集的数目最多是候选集的长度，即2k
est_B=lhb.OLH_aggregation_mutilprocess(epsilon,num_perturbed_B,n_B,2*k,lhb.get_support,8)
# 程序运行结束时间
endtime3 = time.clock()
print('运行时间:', endtime3 - starttime3)
est_B# 项数从0开始



开始执行OLH...
ForkPoolWorker-25 启动
ForkPoolWorker-26 启动
ForkPoolWorker-27 启动
ForkPoolWorker-28 启动
ForkPoolWorker-29 启动
ForkPoolWorker-30 启动
ForkPoolWorker-31 启动
ForkPoolWorker-32 启动
[569, 69, 26, 28, 38, 39, 36, 32, 24, 32, 27, 23, 31, 30, 20, 23, 39, 30, 28, 30, 26, 14, 34, 33, 30, 28, 24, 36, 24, 25, 37, 25, 28, 33, 27, 40, 21, 31, 31, 27]
[535, 68, 33, 25, 23, 30, 32, 30, 25, 38, 37, 29, 29, 33, 22, 34, 35, 27, 34, 34, 29, 29, 28, 28, 29, 30, 36, 35, 28, 22, 27, 27, 26, 29, 33, 37, 26, 35, 28, 38]
[560, 54, 34, 20, 34, 32, 25, 31, 28, 29, 30, 32, 37, 27, 39, 40, 32, 34, 32, 27, 27, 33, 22, 26, 27, 22, 26, 28, 31, 28, 24, 30, 23, 36, 29, 24, 32, 33, 32, 28]
[572, 60, 32, 20, 26, 34, 33, 41, 39, 25, 31, 19, 29, 29, 29, 34, 34, 31, 22, 30, 34, 22, 33, 23, 36, 33, 30, 36, 28, 28, 25, 32, 28, 29, 35, 37, 22, 34, 33, 25]
[539, 85, 36, 28, 33, 24, 27, 19, 32, 24, 30, 37, 37, 38, 23, 34, 34, 38, 26, 25, 21, 20, 28, 36, 34, 29, 39, 26, 26, 28, 28, 30, 29, 33, 19, 30, 34, 36, 24, 28]
[559, 64, 2

[8593.016023318747,
 576.586743613028,
 64.4259840762737,
 -26.669803193109473,
 -2.3775932546072895,
 32.03637082493747,
 -6.42629491102432,
 23.938967512103407,
 -24.645452364900958,
 -10.47499656744135,
 -0.3532424263987743,
 1.6711084018097409,
 17.86591502747786,
 -12.499347395649865,
 -14.52369822385838,
 50.255528278814104,
 46.206826622397074,
 21.91461668389489,
 -28.69415402131799,
 -16.548049052066897,
 -16.548049052066897,
 -69.18117058548829,
 -2.3775932546072895,
 -16.548049052066897,
 17.86591502747786,
 -2.3775932546072895,
 -0.3532424263987743,
 13.817213371060832,
 -42.864609818777595,
 -59.059416444445716,
 -18.572399880275412,
 -44.888960646986106,
 -24.645452364900958,
 30.012019996728956,
 -30.718504849526504,
 38.10942330956301,
 -38.815908162360564,
 56.32858076343965,
 5.719810058226772,
 1.6711084018097409]

##### Step4：获得填充长度返回

In [36]:
#得到填充项长度
padding_length=2

##### GroupC用户完成Step5和6
##### Step5：填充采样并上传

##### (1)GroupC也和Candidate取交集

In [37]:
GroupC_inter=[] #GroupC组用户和Candidate的交集
GroupC_k=[] #GroupC组用户拥有的k值
GroupC_v=[] #GroupC组用户拥有的v值
for i in range(n_C):
    each_k=list(zip(*GroupC[i]))[0]
    temp_k=set(Candidate).intersection(set(each_k))
    GroupC_dict=dict(GroupC[i])
    temp_v=list(map(GroupC_dict.get,temp_k))
    GroupC_inter.append(list(zip(temp_k,temp_v)))
GroupC_inter

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [(142, 1.0)],
 [],
 [],
 [],
 [],
 [(64, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [(11, 1.0)],
 [(48, 1.0), (11, 0.6)],
 [(67, 1.0)],
 [],
 [],
 [],
 [(1, 0.6)],
 [],
 [],
 [(17, 1.0)],
 [],
 [(7, 1.0)],
 [(20, 1.0)],
 [(17, 1.0)],
 [(31, 0.6)],
 [],
 [(49, 0.2)],
 [],
 [],
 [],
 [(1314, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [(101, 0.6)],
 [],
 [(93, 1.0)],
 [(142, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [(49, 0.2)],
 [],
 [],
 [],
 [(3, 1.0)],
 [],
 [(28, 1.0)],
 [],
 [(3, 0.2), (4, 0.6)],
 [],
 [],
 [],
 [],
 [(1731, 1.0), (30, 0.6)],
 [(2, 1.0)],
 [(30, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [],
 [(20, 1.0)],
 [],
 [(8, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [],
 [(2, 0.6), (11, 1.0)],
 [],
 [],
 [],
 [],
 [(28, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [(6, 0.6)],
 [],
 [],
 [(8, 1.0)],
 [(17, 0.6)],
 [(2, 1.0)],
 [],
 [(17, 1.0)],
 [],
 [(1731, 0.2), (21, 1.0), (6, 1.0)],
 [],
 [],
 [],
 [],
 [],
 [(8,

##### (2)Padding and Sampling

In [38]:
pad_item=[i+1 for i in range(d,d+padding_length)]
pad_label=Candidate+pad_item
PS_C=[mlkvp.P_S(GroupC_inter[i],d,padding_length) for i in range(n_C)]
PS_C

[(5852, 1),
 (5852, 1),
 (5852, 1),
 (5851, 1),
 (5852, 1),
 (5851, -1),
 (5852, -1),
 (5852, -1),
 (5851, -1),
 (5851, -1),
 (5852, -1),
 (5851, -1),
 (5851, 1),
 (5852, -1),
 (5851, -1),
 (64, 1),
 (5851, -1),
 (5851, 1),
 (5852, 1),
 (5851, 1),
 (5851, -1),
 (11, 1),
 (48, 1),
 (5851, 1),
 (5852, 1),
 (5851, 1),
 (5851, 1),
 (5851, -1),
 (5852, -1),
 (5851, -1),
 (5852, -1),
 (5852, -1),
 (7, 1),
 (5852, 1),
 (17, 1),
 (5851, 1),
 (5851, -1),
 (5852, 1),
 (5852, 1),
 (5851, 1),
 (5852, 1),
 (5852, -1),
 (5851, 1),
 (5852, 1),
 (5852, -1),
 (5852, 1),
 (5852, 1),
 (5851, -1),
 (5851, 1),
 (5851, 1),
 (5851, 1),
 (5851, 1),
 (5852, -1),
 (101, 1),
 (5852, -1),
 (5851, 1),
 (5852, -1),
 (5851, -1),
 (5852, 1),
 (5852, 1),
 (5851, 1),
 (5852, -1),
 (5851, 1),
 (5851, 1),
 (5851, 1),
 (5851, -1),
 (5852, -1),
 (5851, 1),
 (5851, 1),
 (5851, -1),
 (5852, 1),
 (5851, 1),
 (5852, -1),
 (5852, 1),
 (28, 1),
 (5852, -1),
 (3, -1),
 (5852, -1),
 (5852, -1),
 (5852, -1),
 (5852, 1),
 (1731, 1),

##### (3)UE扰动

In [39]:
eps1=np.log((np.exp(epsilon)+1)/2)
eps2=epsilon
a=0.5
p=lkvb.p_values(eps2)
b=1-lkvb.p_values(eps1)
perturbed_C=mlkvp.PCKV_UE(PS_C,2*k,padding_length,pad_label,a,p,b)
perturbed_C

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

##### Step6：估计频率和均值

In [40]:
f_k,m_k=mlkvp.AEC_UE(perturbed_C,2*k,padding_length,n_C,a,p,b)
print(f_k)
print(m_k)

[0.016364325567077113, 0.013912581994899044, 0.011460838422720989, 0.011460838422720989, 0.008542096074889963, 0.012861834749679878, 0.010643590565328305, 0.0073745991357575492, 0.013095334137506359, 0.010176591789675341, 0.0090090948505429265, 0.0056233537270589315, 0.0079583476053237522, 0.0044558567879265246, 0.0046893561757530055, 0.0073745991357575492, 0.001537114440095492, 0.0058568531148854132, 0.0069076003601045866, 0.0042223574001000429, 0.0060903525027118941, 0.0028213610731411469, 0.006440601584451624, 0.0034051095427073508, 0.008542096074889963, 0.0078415979114105118, 0.0018873635218352213, 0.0024711119914014247, 0.0048061058696662468, 0.0067908506661913462, 0.0094760936261958899, 0.010176591789675341, 0.0080750972992369944, 0.0029381107670543877, 0.0044558567879265246, 0.0030548604609676286, 0.006440601584451624, 0.0073745991357575492, 2.8416356454775368e-05, 0.0051563549514059689]
[1.0349043247097953, 0.46372580983681078, 1.2364304722633681, 1.0755939880665071, 1.26778141

In [41]:
# est_f=[] #估计频率的排名
# for i in range(2*k):
#     est_f.append((Candidate[i],f_k[i]))
# est_f.sort(key=lambda x:x[1],reverse=True)
# est_k=dict(est_f) # 估计频率字典
# est_m=dict(zip(Candidate,m_k)) #估计均值字典
# print('est_k:',est_k)
# print('est_m:',est_m)


est=[] #估计结果按k频率排序
for i in range(2*k):
    est.append((Candidate[i],f_k[i],m_k[i]))
est.sort(key=lambda x:x[1],reverse=True)

est_f={} #频率估计值字典
est_m={} #均值估计值字典
Candidate_rank=[] #按照频率排序的候选集id
for i in range(2*k):
    est_f[est[i][0]]=est[i][1]
    est_m[est[i][0]]=est[i][2]
    Candidate_rank.append(est[i][0])
print('est_f:',est_f)
print('est_m:',est_m)
print("Candidate_rank:",Candidate_rank)

est_f: {8: 0.016364325567077113, 48: 0.013912581994899044, 28: 0.013095334137506359, 101: 0.012861834749679878, 21: 0.011460838422720989, 2: 0.011460838422720989, 30: 0.010643590565328305, 6: 0.010176591789675341, 1973: 0.010176591789675341, 1731: 0.0094760936261958899, 15: 0.0090090948505429265, 11: 0.008542096074889963, 93: 0.008542096074889963, 1: 0.0080750972992369944, 17: 0.0079583476053237522, 1314: 0.0078415979114105118, 98: 0.0073745991357575492, 3: 0.0073745991357575492, 4: 0.0073745991357575492, 67: 0.0069076003601045866, 49: 0.0067908506661913462, 64: 0.006440601584451624, 4410: 0.006440601584451624, 32: 0.0060903525027118941, 13: 0.0058568531148854132, 18: 0.0056233537270589315, 24: 0.0051563549514059689, 40: 0.0048061058696662468, 89: 0.0046893561757530055, 31: 0.0044558567879265246, 3236: 0.0044558567879265246, 20: 0.0042223574001000429, 142: 0.0034051095427073508, 3867: 0.0030548604609676286, 16: 0.0029381107670543877, 38: 0.0028213610731411469, 57: 0.0024711119914014247

##### 获取实际结果

In [42]:

print(true_count)
true_f={} #真实频率字典
true_k={} #真实的项目（不用除以n）
true_m={} #真实均值字典
true_data=[] # 真实频率排名元组list(id,k_count,v_mean)
true_rank=[] # 真实的频率排名
for i in range(d):
    true_k[i+1]=true_count[i][1]
    true_f[i+1]=true_count[i][1]/n
    true_m[i+1]=true_count[i][2]
    true_data.append((i+1,true_count[i][1],true_count[i][2]))

true_data.sort(key=lambda x:x[1],reverse=True)
for i in range(d):
    true_rank.append(true_data[i][0])
print("true_data:",true_data)
print("true_f:",true_f)
print("true_k:",true_k)
print("true_m:",true_m)
print("true_rank:",true_rank)

[[123373.0, 686.0, 0.76034], [123793.0, 1714.0, 0.90758], [124204.0, 801.0, 0.8477], [124553.0, 493.0, 0.71278], [125424.0, 466.0, 0.79914], [125465.0, 883.0, 0.87406], [125564.0, 612.0, 0.75948], [126335.0, 2241.0, 0.78402], [127081.0, 266.0, 0.75338], [127495.0, 460.0, 0.65652], [127865.0, 1393.0, 0.88858], [128730.0, 333.0, 0.78378], [128959.0, 516.0, 0.85892], [129831.0, 210.0, 0.74286], [130259.0, 914.0, 0.75186], [130727.0, 431.0, 0.72994], [131117.0, 981.0, 0.80672], [131533.0, 1091.0, 0.87462], [131698.0, 370.0, 0.83352], [132135.0, 427.0, 0.85762], [132738.0, 1582.0, 0.8407], [133584.0, 208.0, 0.76346], [134015.0, 440.0, 0.82818], [134393.0, 396.0, 0.83636], [134849.0, 312.0, 0.77436], [135459.0, 155.0, 0.68516], [135750.0, 386.0, 0.7886], [136110.0, 1197.0, 0.8162], [136860.0, 708.0, 0.84972], [137585.0, 1100.0, 0.852], [138431.0, 488.0, 0.85246], [139086.0, 524.0, 0.81756], [139792.0, 129.0, 0.92868], [140321.0, 455.0, 0.79516], [140877.0, 160.0, 0.8025], [141300.0, 229.0, 0

##### 指标

##### 1.命中率

In [43]:
hit_ratio=mlkvp.hit_ratio(Candidate_rank,true_rank,k)
hit_ratio

0.75

##### 2.MSE_f

In [44]:
MSE_f=mlkvp.MSE(Candidate_rank,est_f,true_f,k)
MSE_f

9.749e-06

##### 3.MSE_m

In [45]:
MSE_m=mlkvp.MSE(Candidate_rank,est_m,true_m,k)
MSE_m

0.146016421

In [46]:
# 保存结果
save_path='../MLPKV/data/result.txt'

result=[[data_name,epsilon,padding_length,hit_ratio,MSE_f,MSE_m]]
#bf.savetxt(result,save_path)
result

[['Clothing', 5, 2, 0.75, 9.749e-06, 0.146016421]]